In [32]:
import json
import os
import glob
import pandas as pd
from collections import Counter

In [2]:
pose_dir = "./../../data/temp/skeletons/skeletons_tracked"
object_dir = "./../../data/temp/objects"

### Sanity checks

In [3]:
def get_first_json(path):
    files = [f for f in os.listdir(path) if f.endswith(".json")]
    if not files:
        raise FileNotFoundError(f"No JSON files found in {path}")
    files.sort()
    return os.path.join(path, files[0])

pose_json_path = get_first_json(pose_dir)
object_json_path = get_first_json(object_dir)

print(f"Pose file: {pose_json_path}")
print(f"Object file: {object_json_path}\n")

Pose file: ./../../data/temp/skeletons/skeletons_tracked/trimmed_columpios_cam4-2024-09-25_15_45_09_00-00-04_to_00-00-08_743_tracked.json
Object file: ./../../data/temp/objects/trimmed_columpios_cam4-2024-09-25_15_45_09_00-00-04_to_00-00-08_743_objects.json



In [4]:
with open(pose_json_path, "r") as f:
    pose_data = json.load(f)
with open(object_json_path, "r") as f:
    object_data = json.load(f)

print(f"Loaded {len(pose_data)} pose frames and {len(object_data)} object frames.\n")

Loaded 96 pose frames and 96 object frames.



In [6]:
print("Pose sample:")
print(json.dumps(pose_data[0], indent=2)[:600], "\n")

print("Object sample:")
print(json.dumps(object_data[0], indent=2)[:600], "\n")

Pose sample:
{
  "frame": 0,
  "people": []
} 

Object sample:
{
  "frame": 0,
  "objects": [
    {
      "id": 0,
      "class": "person",
      "bbox": [
        889.5,
        650.5,
        938.5,
        732.5
      ],
      "conf": 0.66015625
    }
  ]
} 



In [7]:
pose_frames = [f.get("frame") for f in pose_data if "frame" in f]
obj_frames = [f.get("frame") for f in object_data if "frame" in f]

if pose_frames and obj_frames:
    common_frames = len(set(pose_frames).intersection(set(obj_frames)))
    print(f"Pose frame range: {pose_frames[0]} – {pose_frames[-1]}")
    print(f"Object frame range: {obj_frames[0]} – {obj_frames[-1]}")
    print(f"Common frames: {common_frames}/{len(pose_frames)} ({common_frames/len(pose_frames):.1%})\n")
else:
    print("No frame keys found; check JSON structure.\n")

Pose frame range: 0 – 95
Object frame range: 0 – 95
Common frames: 96/96 (100.0%)



### Check objects detected

In [15]:
json_files = [os.path.join(object_dir, f) for f in os.listdir(object_dir) if f.endswith(".json")]
print(f"Found {len(json_files)} object JSON files\n")

Found 1146 object JSON files



In [20]:
all_classes = []

for file in json_files:
    with open(file, "r") as f:
        data = json.load(f)
    for frame in data:
        for obj in frame.get("objects", []):
            label = obj.get("class", "unknown").strip().lower()
            all_classes.append(label)

In [21]:
counts = Counter(all_classes)
df_classes = pd.DataFrame(counts.items(), columns=["class", "count"]).sort_values("count", ascending=False)

print("All detected object classes:\n")
print(df_classes)
print(f"\nTotal unique classes: {len(df_classes)}")

All detected object classes:

             class   count
0           person  286665
6            bench   85596
1         backpack   11272
13           chair    4759
16             car    4467
2          handbag    4319
28         bicycle    2965
21             dog    2327
19        airplane    1966
7    traffic light    1677
8     fire hydrant    1595
3       skateboard    1388
9            truck    1011
24   parking meter     936
23    dining table     721
18        umbrella     639
15             bus     363
4      sports ball     352
14       surfboard     236
38           donut     198
31       stop sign     188
42        suitcase     182
5    tennis racket     166
37            cake     157
20           train     152
27           clock     135
41          toilet     131
10        elephant     120
17            bird      93
11           horse      78
33         frisbee      73
44    baseball bat      73
12            skis      60
32      motorcycle      56
25             cat      4

In [ ]:
VALID_CLASSES = [
    "bench", "skateboard", "bicycle", "dog", "sports ball", "tennis racket", "frisbee", "chair"
]

### Merge objects into pose-tracking files

In [30]:
pose_dir = "./../../data/temp/skeletons/skeletons_tracked"
object_dir = "./../../data/temp/objects"
output_dir = "./../../data/temp/merged"
os.makedirs(output_dir, exist_ok=True)

In [33]:
pose_files = sorted(glob.glob(os.path.join(pose_dir, "*_tracked.json")))

In [34]:
for pose_path in pose_files:
    base_name = os.path.basename(pose_path).replace("_tracked.json", "")
    obj_path = os.path.join(object_dir, base_name + "_objects.json")
    out_path = os.path.join(output_dir, base_name + "_merged.json")

    if not os.path.exists(obj_path):
        print(f"[Skipping] {base_name}: no matching objects file")
        continue

    with open(pose_path) as f:
        pose_data = json.load(f)
    with open(obj_path) as f:
        obj_data = json.load(f)

    objects_by_frame = {f["frame"]: f.get("objects", []) for f in obj_data}

    merged, total = [], 0
    for f in pose_data:
        frame_id = f["frame"]
        people = f.get("people", [])
        objs = []
        for o in objects_by_frame.get(frame_id, []):
            cls = o.get("class", "").lower()
            if cls in VALID_CLASSES:
                x1, y1, x2, y2 = o["bbox"]
                cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
                objs.append({"class": cls, "centroid": [cx, cy], "conf": o.get("conf", None)})
                total += 1
        merged.append({"frame": frame_id, "people": people, "objects": objs})

    with open(out_path, "w") as f:
        json.dump(merged, f, indent=2)
    print(f"{base_name}: merged {len(merged)} frames, {total} objects added")

trimmed_columpios_cam4-2024-09-25_15_45_09_00-00-04_to_00-00-08_743: merged 96 frames, 0 objects added
trimmed_columpios_cam4-2024-09-25_17_14_39_00-00-06_to_00-00-10_1451: merged 96 frames, 1 objects added
trimmed_columpios_cam4-2024-09-25_17_23_36_00-00-07_to_00-00-11_712: merged 69 frames, 0 objects added
trimmed_columpios_cam4-2024-09-25_19_48_17_00-00-04_to_00-00-08_1438: merged 96 frames, 7 objects added
trimmed_columpios_cam4-2024-09-26_14_40_50_00-00-06_to_00-00-10_134: merged 96 frames, 0 objects added
trimmed_columpios_cam4-2024-09-26_14_56_30_00-00-04_to_00-00-08_1612: merged 96 frames, 0 objects added
trimmed_columpios_cam4-2024-09-26_14_56_30_00-00-08_to_00-00-12_1611: merged 95 frames, 1 objects added
trimmed_columpios_cam4-2024-09-26_16_03_46_00-00-06_to_00-00-10_915: merged 96 frames, 1 objects added
trimmed_columpios_cam4-2024-09-26_16_08_21_00-00-06_to_00-00-10_716: merged 89 frames, 0 objects added
trimmed_columpios_cam4-2024-09-27_13_05_01_00-00-09_to_00-00-13_38: m

In [42]:
merged_dir = "./../../data/temp/merged"
records = []

for f in os.listdir(merged_dir):
    if not f.endswith("_merged.json"):
        continue
    with open(os.path.join(merged_dir, f)) as jfile:
        data = json.load(jfile)
    for frame in data:
        frame_id = frame.get("frame")
        for obj in frame.get("objects", []):
            records.append({
                "video": f.replace("_merged.json",""),
                "frame": frame_id,
                "class": obj.get("class"),
                "conf": obj.get("conf"),
                "cx": obj["centroid"][0],
                "cy": obj["centroid"][1]
            })

df_objects = pd.DataFrame(records)
df_objects.to_csv("./../../data/merged_objects_detail.csv", index=False)
print("Saved to merged_objects_detail.csv")

Saved to merged_objects_detail.csv


In [37]:
df_objects.head()

,video,frame,class,conf,cx,cy
0,trimmed_columpioscam1-2024-10-03_18_49_53_00-0...,0,dog,0.668945,1950.00,512.00
1,trimmed_columpioscam1-2024-10-03_18_49_53_00-0...,0,bench,0.457031,1138.00,1046.25
2,trimmed_columpioscam1-2024-10-03_18_49_53_00-0...,0,bench,0.310791,1091.75,1018.00
3,trimmed_columpioscam1-2024-10-03_18_49_53_00-0...,0,bench,0.307373,854.00,1098.00
4,trimmed_columpioscam1-2024-10-03_18_49_53_00-0...,0,bench,0.259766,1436.00,369.00


In [39]:
df_objects.groupby("video")["class"].count()

video
trimmed_columpios_cam4-2024-09-25_17_14_39_00-00-06_to_00-00-10_1451      1
trimmed_columpios_cam4-2024-09-25_19_48_17_00-00-04_to_00-00-08_1438      7
trimmed_columpios_cam4-2024-09-26_14_56_30_00-00-08_to_00-00-12_1611      1
trimmed_columpios_cam4-2024-09-26_16_03_46_00-00-06_to_00-00-10_915       1
trimmed_columpios_cam4-2024-09-28_16_32_02_00-00-13_to_00-00-17_373     139
                                                                       ... 
trimmed_hundidocam1-2024-10-03_19_04_32_00-00-19_to_00-00-23_203        162
trimmed_hundidocam1-2024-10-03_19_07_09_00-00-06_to_00-00-10_1478       249
trimmed_hundidocam2-2024-10-24_19_12_21_00-00-08_to_00-00-12_193        185
trimmed_hundidocam2-2024-10-24_19_37_59_00-00-13_to_00-00-17_602         21
trimmed_hundidocam2-2024-10-24_19_45_05_00-00-06_to_00-00-10_1460       184
Name: class, Length: 729, dtype: int64

In [40]:
df_objects["class"].value_counts()

class
bench            85596
chair             4759
car               4467
bicycle           2965
dog               2327
skateboard        1388
sports ball        352
tennis racket      166
frisbee             73
Name: count, dtype: int64

In [41]:
df_objects.groupby("class")["conf"].mean()

class
bench            0.464985
bicycle          0.534121
car              0.329160
chair            0.406325
dog              0.584661
frisbee          0.314125
skateboard       0.357138
sports ball      0.596518
tennis racket    0.349962
Name: conf, dtype: float64